In [ ]:
#### IMPORTS ####
include("source/main.jl")
const SIZE = (750,300)

In [ ]:
manager(23, 0.1, collect(0:-0.02:-0.12), ["p", "p"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"], 2.)
manager(43, 0.1, collect(0:-0.02:-0.12), ["p", "p"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"], 2.)
manager(43, 0.1, collect(0:-0.02:-0.12), ["p", "p"], ["scattProb", "kondoCoupFSMap", "kondoCoupDiagMap"], 2.)